# 📘 Архитектуры Агентов 12: Graph / World-Model Memory (Граф / Память Модели Мира)

Добро пожаловать в детальное исследование одной из самых мощных структур памяти для ИИ-агентов: **Graph-based World Model** (Модель Мира на основе Графа). Эта архитектура выходит за рамки простого поиска документов или истории чата, чтобы построить структурированное, взаимосвязанное понимание мира, подобно семантической памяти человека.

Вместо хранения информации в виде изолированных кусков текста, агент на основе графа парсит входящие данные в **сущности (entities / nodes)** и **отношения (relationships / edges)**. Это создает богатый граф знаний (knowledge graph), к которому можно делать запросы. Агент может затем отвечать на сложные вопросы, обходя этот граф и открывая инсайты, которые были бы скрыты в неструктурированном тексте.

Чтобы продемонстрировать это в деталях, мы создадим **Corporate Intelligence Agent** (Агента корпоративной разведки). Этот агент будет:
1.  **Поглощать неструктурированные отчеты:** Читать текстовые документы о компаниях, людях и продуктах.
2.  **Строить Граф Знаний:** Использовать LLM для извлечения сущностей (например, `Company`, `Person`) и отношений (например, `ACQUIRED`, `WORKS_FOR`, `COMPETES_WITH`) и наполнять ими графовую базу данных Neo4j.
3.  **Отвечать на сложные, многоходовые вопросы (Multi-Hop Questions):** Использовать граф для ответа на вопросы, требующие соединения нескольких кусочков информации, например, "*Кто работает в компании, которая приобрела BetaSolutions?*" — запрос, который чрезвычайно сложен для стандартного векторного поиска.

### Определение
**Graph / World-Model Memory** — это агентная архитектура, где знания хранятся в структурированной графовой базе данных. Информация представлена в виде узлов (сущности, такие как люди, места, концепции) и ребер (отношения между ними). Это создает динамическую "модель мира", над которой агент может рассуждать.

### Высокоуровневый рабочий процесс

1.  **Поглощение информации (Information Ingestion):** Агент получает неструктурированные или полуструктурированные данные (текст, документы, ответы API).
2.  **Извлечение знаний (Knowledge Extraction):** Процесс на базе LLM парсит информацию, идентифицируя ключевые сущности и отношения, которые их связывают.
3.  **Обновление Графа (Graph Update):** Извлеченные узлы и ребра добавляются или обновляются в персистентной графовой базе данных (например, Neo4j).
4.  **Ответы на вопросы / Рассуждение (Question Answering / Reasoning):** Когда задается вопрос, агент:
    a. Конвертирует вопрос на естественном языке в формальный язык запросов к графу (например, Cypher для Neo4j).
    b. Выполняет запрос к графу для получения релевантных подграфов или фактов.
    c. Синтезирует результаты запроса в ответ на естественном языке.

### Когда использовать / Применения
*   **Корпоративные ассистенты по знаниям:** Построение модели проектов компании, сотрудников и клиентов из внутренних документов, к которой можно делать запросы.
*   **Продвинутые ассистенты-исследователи:** Создание динамической базы знаний научной области путем поглощения исследовательских статей.
*   **Диагностика сложных систем:** Моделирование компонентов системы и их зависимостей для диагностики сбоев.

### Сильные и слабые стороны
*   **Сильные стороны:**
    *   **Структурированность и Объяснимость:** Знания высоко организованы. Ответ может быть объяснен демонстрацией точного пути в графе, который к нему привел.
    *   **Обеспечивает сложное рассуждение:** Превосходно отвечает на "многоходовые" (multi-hop) вопросы, требующие соединения разрозненных частей информации через отношения.
*   **Слабые стороны:**
    *   **Начальная сложность:** Требует хорошо определенной схемы и надежного процесса извлечения.
    *   **Поддержание актуальности графа:** Может быть сложным управлять обновлениями, разрешать противоречивую информацию и удалять устаревшие факты со временем (управление жизненным циклом знаний).

## Фаза 0: Основа и Настройка

Мы установим библиотеки, включая драйвер Neo4j, и настроим нашу среду. **Критически важно: у вас должен быть запущен экземпляр Neo4j и файл `.env` с учетными данными к нему.**

In [ ]:
import os
import json
from typing import List, Annotated, TypedDict, Optional
from dotenv import load_dotenv

# Pydantic для структурированного вывода
# ПРИМЕЧАНИЕ: Мы используем BaseModel из langchain_core.pydantic_v1 для лучшей совместимости с структурированным выводом на данный момент
from langchain_core.pydantic_v1 import BaseModel as V1BaseModel, Field

# Компоненты LangChain
from langchain_nebius import ChatNebius, NebiusEmbeddings
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector

# Компоненты LangGraph (не строго требуются для Фазы 1, но полезны для агента в Фазе 3)
from langgraph.graph import StateGraph, END

# Для красивого вывода
from rich.console import Console
from rich.markdown import Markdown

from notebook_utils import setup_environment

# --- Настройка API ключей и трассировки ---
console = setup_environment(
    project_name="Agentic Architecture - GraphRAG (Nebius)",
    required_keys=["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "NEO4J_URI", "NEO4J_USERNAME", "NEO4J_PASSWORD"]
)

In [ ]:
import os
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# Pydantic для моделирования данных
from pydantic import BaseModel, Field

# Компоненты LangChain
from langchain_nebius import ChatNebius
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel as V1BaseModel

# Для красивого вывода
from rich.console import Console
from rich.markdown import Markdown

# --- Настройка API ключей и трассировки ---
load_dotenv(dotenv_path="../.env")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Graph Memory (Nebius)"

# Проверка необходимых переменных окружения
required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "NEO4J_URI", "NEO4J_USERNAME", "NEO4J_PASSWORD"]
for var in required_vars:
    if var not in os.environ:
        print(f"Warning: Environment variable {var} not set.")

print("Переменные окружения загружены и трассировка настроена.")

Environment variables loaded and tracing is set up.


## Фаза 1: Построение Агента Создания Графа (Graph Construction Agent)

Это сердце пайплайна поглощения данных. Нам нужен агент, который может читать неструктурированный текст и извлекать сущности и отношения в структурированном формате. Мы будем использовать LLM с возможностями структурированного вывода (Pydantic), чтобы он действовал как наш экстрактор знаний.

In [ ]:
console = Console()
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0)

# Подключение к нашей базе данных Neo4j
try:
    graph = Neo4jGraph()
    # Очистка графа для чистого запуска
    graph.query("MATCH (n) DETACH DELETE n")
except Exception as e:
    console.print(f"[bold red]Не удалось подключиться к Neo4j: {e}. Пожалуйста, проверьте учетные данные и соединение.[/bold red]")
    graph = None

# Pydantic модели для структурированного извлечения (используем v1 BaseModel из LangChain для совместимости)
class Node(V1BaseModel):
    id: str = Field(description="Уникальное имя или идентификатор сущности.")
    type: str = Field(description="Тип или метка сущности (например, Person, Company, Product).")

class Relationship(V1BaseModel):
    source: Node
    target: Node
    type: str = Field(description="Тип отношения (например, WORKS_FOR, ACQUIRED).")

class KnowledgeGraph(V1BaseModel):
    """Граф узлов и отношений."""
    relationships: List[Relationship]

# Агент создания графа (The Graph Maker Agent)
def get_graph_maker_chain():
    extractor_llm = llm.with_structured_output(KnowledgeGraph)
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an expert at extracting information from text and building a knowledge graph. Extract all entities (nodes) and relationships from the provided text. The relationship type should be a verb in all caps, like 'WORKS_FOR' or 'ACQUIRED'."),
        ("human", "Extract a knowledge graph from the following text:\n\n{text}")
    ])
    return prompt | extractor_llm

graph_maker_agent = get_graph_maker_chain()
print("Успешно подключились к Neo4j и определили Агента Создания Графа.")

Successfully connected to Neo4j and defined the Graph Maker Agent.


## Фаза 2: Поглощение Знаний и Построение Модели Мира

Теперь мы "скормим" нашему агенту серию связанных, но отдельных документов. Агент обработает каждый из них и постепенно построит наш корпоративный граф знаний. Это симулирует то, как реальная система училась бы со временем по мере поступления новой информации.

In [ ]:
unstructured_documents = [
    "On May 15, 2023, global tech giant AlphaCorp announced its acquisition of startup BetaSolutions, a leader in cloud-native database technology.",
    "Dr. Evelyn Reed, a renowned AI researcher, has been the Chief Science Officer at AlphaCorp since 2021. She leads the team responsible for the QuantumLeap AI platform.",
    "Innovate Inc.'s flagship product, NeuraGen, is seen as a direct competitor to AlphaCorp's QuantumLeap AI. Meanwhile, Innovate Inc. recently hired Johnathan Miles as their new CTO."
]
for i, doc in enumerate(unstructured_documents):
    console.print(f"--- Поглощение Документа {i+1} ---")
    try:
        kg_data = graph_maker_agent.invoke({"text": doc})
        if kg_data.relationships:
            graph.add_graph_documents(graph_documents=kg_data.relationships, include_source=False)
            console.print(f"[green]Успешно добавлено {len(kg_data.relationships)} отношений в граф.[/green]")
        else:
             console.print("[yellow]Отношения не извлечены.[/yellow]")
    except Exception as e:
        console.print(f"[red]Не удалось обработать документ: {e}[/red]")

console.print("--- ✅ Поглощение Графа Знаний Завершено ---")
console.print("\n--- Схема Графа (Schema) ---")
console.print(graph.schema)

--- Ingesting Document 1 ---
Successfully added 1 relationships to the graph.
--- Ingesting Document 2 ---
Successfully added 2 relationships to the graph.
--- Ingesting Document 3 ---
Successfully added 2 relationships to the graph.
--- ✅ Knowledge Graph Ingestion Complete ---



--- Graph Schema ---


Node properties: [{'properties': [('id', 'STRING')], 'labels': ['Product']}, {'properties': [('id', 'STRING')], 'labels': ['Person']}, {'properties': [('id', 'STRING')], 'labels': ['Company']}]
Relationship properties: []
Relationships: [(:Company)-[:PRODUCES]->(:Product), (:Person)-[:WORKS_FOR]->(:Company), (:Product)-[:COMPETES_WITH]->(:Product), (:Company)-[:ACQUIRED]->(:Company)]


## Фаза 3: Построение Агента Запросов к Графу (Graph-Querying Agent)

С наполненным графом знаний нам нужен агент, который может его использовать. Это включает в себя процесс **Text-to-Cypher** (Текст-в-Cypher). Агент получит вопрос пользователя на естественном языке, конвертирует его в запрос Cypher, используя схему графа как контекст, выполнит запрос, а затем синтезирует результаты в читаемый человеком ответ.

In [ ]:
# LangChain имеет встроенную цепочку для этого, но мы рассмотрим компоненты,
# чтобы понять, как это работает.
cypher_generation_prompt = ChatPromptTemplate.from_template(
    """You are an expert Neo4j Cypher query developer. Your task is to convert a user's natural language question into a valid Cypher query.
You must use the provided graph schema to construct the query. Do not use any node labels or relationship types that are not in the schema.
Return ONLY the Cypher query, with no additional text or explanations.

Graph Schema:
{schema}

User Question:
{question}
"""
)

cypher_response_prompt = ChatPromptTemplate.from_template(
    """You are an assistant that provides clear, natural language answers based on query results from a knowledge graph.
Use the context from the graph query result to answer the user's original question.

User Question: {question}
Query Result: {context}
"""
)

def query_graph(question: str) -> Dict[str, Any]:
    """Полный пайплайн Text-to-Cypher и синтеза."""
    console.print(f"\n[bold]Вопрос:[/bold] {question}")
    
    # 1. Генерация Cypher запроса
    console.print("--- ➡️ Генерация Cypher Запроса ---")
    cypher_chain = cypher_generation_prompt | llm
    generated_cypher = cypher_chain.invoke({"schema": graph.schema, "question": question}).content
    console.print(f"[cyan]Сгенерированный Cypher:\n{generated_cypher}[/cyan]")
    
    # 2. Выполнение Cypher запроса
    console.print("--- ⚡ Выполнение Запроса ---")
    try:
        context = graph.query(generated_cypher)
        console.print(f"[yellow]Результат Запроса:\n{context}[/yellow]")
    except Exception as e:
        console.print(f"[red]Cypher запрос не удался: {e}[/red]")
        return {"answer": "I was unable to execute a query to find the answer to your question."}
    
    # 3. Синтез Финального Ответа
    console.print("--- 🗣️ Синтез Финального Ответа ---")
    synthesis_chain = cypher_response_prompt | llm
    answer = synthesis_chain.invoke({"question": question, "context": context}).content
    
    return {"answer": answer}

print("Агент Запросов к Графу успешно определен.")

Graph-Querying Agent defined successfully.


## Фаза 4: Демонстрация и Анализ

Теперь финальный тест. Мы зададим нашему агенту вопросы, варьирующиеся от простого извлечения фактов до сложного, многоходового рассуждения, которое требует соединения информации из всех трех наших исходных документов.

In [ ]:
# Тест 1: Простое извлечение факта (требует инфо из документа 2)
result1 = query_graph("Who works for AlphaCorp?")
console.print("\n--- Финальный ответ ---")
console.print(Markdown(result1['answer']))

# Тест 2: Другое простое извлечение факта (требует инфо из документа 1)
result2 = query_graph("What company did AlphaCorp acquire?")
console.print("\n--- Финальный ответ ---")
console.print(Markdown(result2['answer']))

# Тест 3: Вопрос на многоходовое рассуждение (требует инфо из всех 3 документов)
result3 = query_graph("What companies compete with the products made by the company that acquired BetaSolutions?")
console.print("\n--- Финальный ответ ---")
console.print(Markdown(result3['answer']))


Question: Who works for AlphaCorp?
--- ➡️ Generating Cypher Query ---
Generated Cypher:
MATCH (p:Person)-[:WORKS_FOR]->(c:Company {id: 'AlphaCorp'}) RETURN p.id
--- ⚡ Executing Query ---
Query Result:
[{'p.id': 'Dr. Evelyn Reed'}]
--- 🗣️ Synthesizing Final Answer ---



--- Final Answer ---


Dr. Evelyn Reed works for AlphaCorp.


Question: What company did AlphaCorp acquire?
--- ➡️ Generating Cypher Query ---
Generated Cypher:
MATCH (:Company {id: 'AlphaCorp'})-[:ACQUIRED]->(acquired_company:Company)
RETURN acquired_company.id
--- ⚡ Executing Query ---
Query Result:
[{'acquired_company.id': 'BetaSolutions'}]
--- 🗣️ Synthesizing Final Answer ---



--- Final Answer ---


AlphaCorp acquired BetaSolutions.


Question: What companies compete with the products made by the company that acquired BetaSolutions?
--- ➡️ Generating Cypher Query ---
Generated Cypher:
MATCH (acquirer:Company)-[:ACQUIRED]->(:Company {id: 'BetaSolutions'})
MATCH (acquirer)-[:PRODUCES]->(product:Product)
MATCH (product)-[:COMPETES_WITH]->(competitor_product:Product)
MATCH (competitor_company:Company)-[:PRODUCES]->(competitor_product)
RETURN DISTINCT competitor_company.id
--- ⚡ Executing Query ---
Query Result:
[{'competitor_company.id': 'Innovate Inc.'}]
--- 🗣️ Synthesizing Final Answer ---



--- Final Answer ---


Innovate Inc. competes with the products made by the company that acquired BetaSolutions.

### Анализ результатов

Демонстрация подчеркивает глубокое преимущество модели мира на основе графов:

- Первые два вопроса были простыми поисками. Агент успешно конвертировал вопросы в Cypher, запросил граф и нашел прямые отношения.

- Третий вопрос — решающий. Стандартный RAG агент здесь бы провалился. Он мог бы найти документ о приобретении и документ о конкуренте, но ему было бы трудно соединить их. У него отсутствует явная реляционная структура, чтобы понять, что "AlphaCorp" в документе 1 — это та же сущность, что и "AlphaCorp" в документах 2 и 3.

- Наш агент на графах, однако, решил это с легкостью. Мы можем проследить его логику прямо из сгенерированного запроса Cypher:
    1.  `MATCH (acquirer:Company)-[:ACQUIRED]->(:Company {id: 'BetaSolutions'})`: Сначала найти компанию, которая приобрела BetaSolutions (Результат: AlphaCorp).
    2.  `MATCH (acquirer)-[:PRODUCES]->(product:Product)`: Затем найти продукты, производимые этой компанией (Результат: QuantumLeap AI).
    3.  `MATCH (product)-[:COMPETES_WITH]->(competitor_product:Product)`: Затем найти продукты, которые конкурируют с этим продуктом (Результат: NeuraGen).
    4.  `MATCH (competitor_company:Company)-[:PRODUCES]->(competitor_product)`: Наконец, найти компанию, которая производит конкурирующий продукт (Результат: Innovate Inc.).

Эта способность проходить по отношениям и синтезировать информацию из разных источников является суперспособностью архитектуры Graph / World-Model. Ответ не просто извлекается; он выводится рассуждением.

## Заключение

В этом блокноте мы построили полную агентную систему, построенную вокруг **Graph / World-Model Memory** (Памяти Модели Мира на Графах). Мы продемонстрировали полный жизненный цикл: поглощение неструктурированных данных, использование LLM для построения структурированного графа знаний, а затем использование этого графа для ответа на сложные вопросы с многоходовым рассуждением.

Эта архитектура представляет собой значительный скачок в возможностях по сравнению с более простыми системами памяти. Создавая явную модель мира с возможностью запросов, мы даем нашим агентам способность соединять разрозненные факты и открывать скрытые инсайты. Хотя вызовы поддержания этого графа во времени реальны, потенциал для построения глубоко осведомленных и объяснимых ИИ-ассистентов делает это одним из самых захватывающих и мощных паттернов в современном агентном дизайне.